In [1]:
import pandas as pd
import numpy as np
import difflib

In [2]:
# creating lists to use for later looping
transfer_data = ["italian","spanish","english"]
wage_data = ["SerieA","Liga","EPl"]

In [3]:
# Creating a function to join two cleaned datasets
def transfer_joiner(team_a,team_b):
    file_transfer = f"../cleaned/{team_a}_transfers.csv"
    file_wage = f"../../Team_player/cleaned/{team_b}.csv"
    transfer = pd.read_csv(file_transfer)
    transfer = transfer.rename(columns={"year":"Year"})
    transfer = transfer[["Year","Teams","Transfer Spend (Euros)","MA"]]
    wage = pd.read_csv(file_wage)
    wage = wage[["Year","Teams", "Ranking","avg player value (EU)","avg player wage"]]
    # As certain team names appear different in the sets, this line does a 'fuzzy' lookup to match the names from one dataframe to another
    wage["Teams"] = wage["Teams"].apply(lambda x: difflib.get_close_matches(x,transfer["Teams"])[0])
    wage["Teams"] = wage["Teams"].str.replace("FC Internazionale","Inter Milan")
    joined = pd.merge(wage,transfer,on=["Year","Teams"],how="outer")
    joined = joined[joined["Year"]>2014]
    return joined

In [33]:
italy_joined = transfer_joiner(transfer_data[0],wage_data[0])
spain_joined = transfer_joiner(transfer_data[1],wage_data[1])
england_joined = transfer_joiner(transfer_data[2],wage_data[2])

In [16]:
def top4Clean(df, country):
    df = df[["Year","Teams","Ranking","avg player value (EU)","avg player wage","Transfer Spend (Euros)"]]
    df = df.rename(columns={"Teams":"Team","avg player value (EU)":"Average_Player_Value","avg player wage":"Player_Wages","Transfer Spend (Euros)":"Transfer_spend"})
    df["League"]=country
    df = df[df["Ranking"]<5]
    return df

In [21]:
italy_top4 = top4Clean(italy_joined,"Italy")
spain_top4 = top4Clean(spain_joined,"Spain")
england_top4 = top4Clean(england_joined,"England")
top_4 = pd.concat([italy_top4,spain_top4,england_top4])

In [ ]:
italy_joined.to_csv("../cleaned/italy_joined.csv")
spain_joined.to_csv("../cleaned/spain_joined.csv")
england_joined.to_csv("../cleaned/england_joined.csv")

In [22]:
top_4.to_csv("../cleaned/top4_financials.csv")

In [54]:
def summariser(joined_df,group,league):
    years = [2015,2016,2017,2018,2019]

    league_wage = []
    league_pv = []
    transfers = []
    for year in years:
        join = joined_df[joined_df["Year"]==year]
        join
        league_wage.append(join["avg player wage"].mean())
        league_pv.append(join["avg player value (EU)"].mean())
        transfers.append(join["Transfer Spend (Euros)"].mean())

    dictionary = {"Year":years,"Wages":league_wage,"Value":league_pv,"Transfers":transfers}
    df = pd.DataFrame.from_dict(dictionary)
    df["League"]=league
    df["Group"]=group
    return df

In [65]:
def summaryTable(df, league):

    top4_summary = summariser(df[df["Ranking"]<5],"Top 4",league)
    bottom4_summary = summariser(df[df["Ranking"]>16],"Bottom 4",league)
    league_summary = summariser(df,"League",league)
    summary_df = pd.concat([top4_summary,bottom4_summary,league_summary])
    return summary_df


In [68]:
Italy_df = summaryTable(italy_joined,"Italy")
England_df=summaryTable(england_joined,"England")
Spain_df = summaryTable(spain_joined,"Spain")
summary_df = pd.concat([Italy_df,England_df,Spain_df])


In [69]:
summary_df.to_csv("../cleaned/summary_financials.csv")